In [20]:
import collections
import math

import torch
from torch.utils.data import DataLoader, TensorDataset, SequentialSampler
from pytorch_pretrained_bert.modeling import BertForQuestionAnswering, BertConfig
from pytorch_pretrained_bert.tokenization import BertTokenizer
from pythainlp.tokenize import word_tokenize

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Utils

In [3]:
class Example(object):
    """
    A single example for prediction. a context and a question
    per example.
    """

    def __init__(self, context, question):
        self.context = context
        self.question = question

    def __str__(self):
        return self.__repr__()

    def __repr__(self):
        s = f'Context: {self.context}\n'
        s += f'Question: {question}'
        return s


class InputFeatures(object):
    """A single set of features of data."""

    def __init__(self,
                 unique_id,
                 example_index,
                 doc_span_index,
                 tokens,
                 token_to_orig_map,
                 token_is_max_context,
                 input_ids,
                 input_mask,
                 segment_ids):
        self.unique_id = unique_id
        self.example_index = example_index
        self.doc_span_index = doc_span_index
        self.tokens = tokens
        self.token_to_orig_map = token_to_orig_map
        self.token_is_max_context = token_is_max_context
        self.input_ids = input_ids
        self.input_mask = input_mask
        self.segment_ids = segment_ids


def convert_examples_to_features(examples, tokenizer,
                                 max_seq_length=384, doc_stride=128):
    """Loads a data file into a list of `InputBatch`s."""
    unique_id = 1000001
    features = []
    for example_index, example in enumerate(examples):
        doc_tokens = word_tokenize(example.context)
        question_tokens = tokenizer.tokenize(example.question)
        
        tok_to_orig_index = []
        orig_to_tok_index = []
        all_doc_tokens = []
        for i, token in enumerate(doc_tokens):
            orig_to_tok_index.append(len(all_doc_tokens))
            sub_tokens = tokenizer.tokenize(token)
            for sub_token in sub_tokens:
                tok_to_orig_index.append(i)
                all_doc_tokens.append(sub_token)

        tok_start_position = None
        tok_end_position = None

        # The -3 accounts for [CLS], [SEP] and [SEP]
        max_tokens_for_doc = max_seq_length - len(question_tokens) - 3

        # We can have documents that are longer than the maximum sequence length.
        # To deal with this we do a sliding window approach, where we take chunks
        # of the up to our max length with a stride of `doc_stride`.
        _DocSpan = collections.namedtuple(  # pylint: disable=invalid-name
            "DocSpan", ["start", "length"])
        doc_spans = []
        start_offset = 0
        while start_offset < len(all_doc_tokens):
            length = len(all_doc_tokens) - start_offset
            if length > max_tokens_for_doc:
                length = max_tokens_for_doc
            doc_spans.append(_DocSpan(start=start_offset, length=length))
            if start_offset + length == len(all_doc_tokens):
                break
            start_offset += min(length, doc_stride)

        for doc_span_index, doc_span in enumerate(doc_spans):
            tokens = []
            token_to_orig_map = {}
            token_is_max_context = {}
            segment_ids = []
            tokens.append("[CLS]")
            segment_ids.append(0)
            for token in question_tokens:
                tokens.append(token)
                segment_ids.append(0)
            tokens.append("[SEP]")
            segment_ids.append(0)

            for i in range(doc_span.length):
                split_token_index = doc_span.start + i
                token_to_orig_map[len(tokens)] = tok_to_orig_index[split_token_index]

                is_max_context = _check_is_max_context(doc_spans, doc_span_index,
                                                       split_token_index)
                token_is_max_context[len(tokens)] = is_max_context
                tokens.append(all_doc_tokens[split_token_index])
                segment_ids.append(1)
            tokens.append("[SEP]")
            segment_ids.append(1)

            input_ids = tokenizer.convert_tokens_to_ids(tokens)

            # The mask has 1 for real tokens and 0 for padding tokens. Only real
            # tokens are attended to.
            input_mask = [1] * len(input_ids)

            # Zero-pad up to the sequence length.
            while len(input_ids) < max_seq_length:
                input_ids.append(0)
                input_mask.append(0)
                segment_ids.append(0)

            assert len(input_ids) == max_seq_length
            assert len(input_mask) == max_seq_length
            assert len(segment_ids) == max_seq_length

            start_position = None
            end_position = None

            features.append(
                InputFeatures(
                    unique_id=unique_id,
                    example_index=example_index,
                    doc_span_index=doc_span_index,
                    tokens=tokens,
                    token_to_orig_map=token_to_orig_map,
                    token_is_max_context=token_is_max_context,
                    input_ids=input_ids,
                    input_mask=input_mask,
                    segment_ids=segment_ids))
            unique_id += 1
    return features

def _check_is_max_context(doc_spans, cur_span_index, position):
    """Check if this is the 'max context' doc span for the token."""

    # Because of the sliding window approach taken to scoring documents, a single
    # token can appear in multiple documents. E.g.
    #  Doc: the man went to the store and bought a gallon of milk
    #  Span A: the man went to the
    #  Span B: to the store and bought
    #  Span C: and bought a gallon of
    #  ...
    #
    # Now the word 'bought' will have two scores from spans B and C. We only
    # want to consider the score with "maximum context", which we define as
    # the *minimum* of its left and right context (the *sum* of left and
    # right context will always be the same, of course).
    #
    # In the example the maximum context for 'bought' would be span C since
    # it has 1 left context and 3 right context, while span B has 4 left context
    # and 0 right context.
    best_score = None
    best_span_index = None
    for (span_index, doc_span) in enumerate(doc_spans):
        end = doc_span.start + doc_span.length - 1
        if position < doc_span.start:
            continue
        if position > end:
            continue
        num_left_context = position - doc_span.start
        num_right_context = end - position
        score = min(num_left_context, num_right_context) + 0.01 * doc_span.length
        if best_score is None or score > best_score:
            best_score = score
            best_span_index = span_index

    return cur_span_index == best_span_index

def predict(all_examples, all_features, all_results,
            n_best_size, top_k, max_answer_length):
    """Predict."""

    example_index_to_features = collections.defaultdict(list)
    for feature in all_features:
        example_index_to_features[feature.example_index].append(feature)

    unique_id_to_result = {}
    for result in all_results:
        unique_id_to_result[result.unique_id] = result

    _PrelimPrediction = collections.namedtuple(
        "PrelimPrediction",
        ["feature_index", "start_index", "end_index", "start_logit", "end_logit"])

    all_predictions = collections.OrderedDict()
    all_nbest_json = collections.OrderedDict()
    scores_diff_json = collections.OrderedDict()

    results = []
    for (example_index, example) in enumerate(all_examples):
        features = example_index_to_features[example_index]

        prelim_predictions = []
        # keep track of the minimum score of null start+end of position 0
        score_null = 1000000  # large and positive
        min_null_feature_index = 0  # the paragraph slice with min null score
        null_start_logit = 0  # the start logit at the slice with min null score
        null_end_logit = 0  # the end logit at the slice with min null score


        for (feature_index, feature) in enumerate(features):
            result = unique_id_to_result[feature.unique_id]
            start_indexes = _get_best_indexes(result.start_logits, n_best_size)
            end_indexes = _get_best_indexes(result.end_logits, n_best_size)
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # We could hypothetically create invalid predictions, e.g., predict
                    # that the start of the span is in the question. We throw out all
                    # invalid predictions.
                    if start_index >= len(feature.tokens):
                        continue
                    if end_index >= len(feature.tokens):
                        continue
                    if start_index not in feature.token_to_orig_map:
                        continue
                    if end_index not in feature.token_to_orig_map:
                        continue
                    if not feature.token_is_max_context.get(start_index, False):
                        continue
                    if end_index < start_index:
                        continue
                    length = end_index - start_index + 1
                    if length > max_answer_length:
                        continue
                    prelim_predictions.append(
                        _PrelimPrediction(
                            feature_index=feature_index,
                            start_index=start_index,
                            end_index=end_index,
                            start_logit=result.start_logits[start_index],
                            end_logit=result.end_logits[end_index]))

        prelim_predictions = sorted(
            prelim_predictions,
            key=lambda x: (x.start_logit + x.end_logit),
            reverse=True)

        _NbestPrediction = collections.namedtuple(
            "NbestPrediction", ["text", "start_logit", "end_logit"])

        seen_predictions = {}
        nbest = []
        for pred in prelim_predictions:
            if len(nbest) >= n_best_size:
                break
            feature = features[pred.feature_index]
            if pred.start_index > 0:  # this is a non-null prediction
                tok_tokens = feature.tokens[pred.start_index:(pred.end_index + 1)]
                tok_text = " ".join(tok_tokens)

                # De-tokenize WordPieces that have been split off.
                tok_text = tok_text.replace(" ##", "")
                tok_text = tok_text.replace("##", "")

                # Clean whitespace
                tok_text = tok_text.strip()
                tok_text = " ".join(tok_text.split())

                if tok_text in seen_predictions:
                    continue

                seen_predictions[tok_text] = True
            else:
                tok_text = ""
                seen_predictions[tok_text] = True

            nbest.append(
                _NbestPrediction(
                    text=tok_text,
                    start_logit=pred.start_logit,
                    end_logit=pred.end_logit))
                
            # In very rare edge cases we could only have single null prediction.
            # So we just create a nonce prediction in this case to avoid failure.
            if len(nbest) == 1:
                nbest.insert(0,
                    _NbestPrediction(text="empty", start_logit=0.0, end_logit=0.0))
                
        # In very rare edge cases we could have no valid predictions. So we
        # just create a nonce prediction in this case to avoid failure.
        if not nbest:
            nbest.append(
                _NbestPrediction(text="empty", start_logit=0.0, end_logit=0.0))

        assert len(nbest) >= 1
        total_scores = []
        best_non_null_entry = None
        for entry in nbest:
            total_scores.append(entry.start_logit + entry.end_logit)
            if not best_non_null_entry:
                if entry.text:
                    best_non_null_entry = entry

        probs = _compute_softmax(total_scores)

        nbest_json = []
        for (i, entry) in enumerate(nbest):
            output = collections.OrderedDict()
            output["text"] = entry.text
            output["probability"] = probs[i]
#             output["start_logit"] = entry.start_logit
#             output["end_logit"] = entry.end_logit
            nbest_json.append(output)

        assert len(nbest_json) >= 1

        all_predictions[example_index] = nbest_json[0]["text"]
        
        sorted_nbest_json = sorted(nbest_json, key=lambda x: x['probability'], reverse=True)
        
        results.append((example.question, sorted_nbest_json[:top_k]))

    return results

def _get_best_indexes(logits, n_best_size):
    """Get the n-best logits from a list."""
    index_and_score = sorted(enumerate(logits), key=lambda x: x[1], reverse=True)

    best_indexes = []
    for i in range(len(index_and_score)):
        if i >= n_best_size:
            break
        best_indexes.append(index_and_score[i][0])
    return best_indexes

def _compute_softmax(scores):
    """Compute softmax probability over raw logits."""
    if not scores:
        return []

    max_score = None
    for score in scores:
        if max_score is None or score > max_score:
            max_score = score

    exp_scores = []
    total_sum = 0.0
    for score in scores:
        x = math.exp(score - max_score)
        exp_scores.append(x)
        total_sum += x

    probs = []
    for score in exp_scores:
        probs.append(score / total_sum)
    return probs

# Load checkpoint

In [5]:
# Load checkpoint
checkpoint_dir = '/Users/jeew/works/qas/checkpoint/thai_only'

config = BertConfig.from_json_file(f'{checkpoint_dir}/config.json')
model = BertForQuestionAnswering(config)
state_dict = torch.load(f'{checkpoint_dir}/pytorch_model.bin', map_location=device)
model.load_state_dict(state_dict)
bert_tokenizer = BertTokenizer(f'{checkpoint_dir}/vocab.txt', do_lower_case=False)
model.to(device)

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): BertLayerNorm()
      (dropout): Dropout(p=0.1)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): BertLayerNorm()
              (dropout): Dropout(p=0.1)
            )
          )
          (intermediate): 

# Raw Input

In [75]:
context = 'ณเดชน์ คูกิมิยะ เกิดเมื่อวันที่ 17 ธันวาคม พ.ศ. 2534 เป็นนักแสดงและนายแบบลูกครึ่งไทย-ออสเตรีย มีชื่อเสียงจากผลงานแสดงทางสถานีวิทยุโทรทัศน์ไทยทีวีสีช่อง 3 โดยเฉพาะละครเรื่อง "ดวงใจอัคนี, เกมร้ายเกมรัก" ซึ่งแสดงร่วมกับอุรัสยา เสปอร์บันด์ เป็นผู้นำเสนอภาพยนตร์โฆษณาสินค้าต่าง ๆ มากมาย เข้าร่วมในกิจกรรมรณรงค์ด้านสังคม งานการกุศล เช่น เป็นพรีเซ็นเตอร์ รณรงค์เชิญชวนชายไทยคัดเลือกทหาร ให้กับทางกองทัพบก การได้รับเลือกจากสภากาชาดไทย เป็นทูตรณรงค์เผยแพร่ความรู้ต้านภัยมะเร็งเต้านมในปี 2554-2555 ได้รับเลือกจากมหาวิทยาลัยมหาจุฬาลงกรณราชวิทยาลัย ให้เป็นทูตพระพุทธศาสนาวิสาขบูชานานาชาติ เป็นต้น มีผลงานการพากย์เสียงการ์ตูน และได้รับรางวัลผลงานดีเด่นในวงการบันเทิงมากมาย จากการที่ประชาชนพบเห็นณเดชน์ปรากฏตัวผ่านสื่อต่าง ๆ มากมายอยู่เป็นประจำ ไม่ว่าจะเป็น ละครโทรทัศน์, ภาพยนตร์, โฆษณา, นิตยสาร ฯลฯ ทำให้บางกอกโพสต์ เรียกณเดชน์ว่า "มิสเตอร์เอพวี่แวร์" ความสำเร็จในวงการบันเทิง อาทิเช่น ได้รับฉายา "ซุปตาร์พันธุ์ข้าวเหนียว" จากสมาคมนักข่าวบันเทิงในปี พ.ศ. 2554 การได้รับรางวัล "ขวัญใจมหาชน" จากงาน สยามดาราสตาร์อวอร์ด 2011 รวมทั้งรางวัลนักแสดงนำชายดีเด่นในหลาย ๆ สถาบันที่สำคัญ จากละครเรื่อง "เกมร้ายเกมรัก" ได้แก่ รางวัลเมขลา ครั้งที่ 24, รางวัลนาฏราช ครั้งที่ 3, สยามดาราสตาร์อวอร์ด 2018 เป็นต้น ไปจนถึงได้รับการยกย่องให้เป็น 1 ใน 14 ดาราแห่งศตวรรษ และ 1 ใน 120 คนบันเทิง ผู้ทรงอิทธิพลครึ่งปีแรก 2555 โดยนิตยสารวิเคราะห์วงการบันเทิง รีเควส รวมทั้งดาราทรงอิทธิพลในวงการบันเทิง โดยหนังสือพิมพ์ ดาราเดลี่ เป็นต้น ไปจนถึงผลการสำรวจความนิยมในอันดับ 1 ของสถาบันต่าง ๆ มากมายประวัติ ประวัติ. ณเดชน์ คูกิมิยะ มีชื่อจริงว่า ชลทิศ ยอดประทุม เกิดวันอังคารวันที่ 17 ธันวาคม พ.ศ. 2534 ที่จังหวัดขอนแก่น โดยชื่อจริงมารดาบุญธรรมเป็นผู้ตั้งให้ เป็นคำประสมระหว่าง ณ+เดชน์ มีความหมายว่า "ผู้มีปัญญาเฉลียวฉลาด" โดยความหมายในพจนานุกรม คำว่า "เดชน์" แปลว่า "ลูกศร" ส่วน ณ ในที่นี้มาจากคำว่าญาณ หมายถึง ปัญญา ความรู้ ซึ่งตัด ณ มาเพียงตัวเดียว เมื่อรวมความหมายเข้าด้วยกันแล้ว จะได้ความหมายว่า ผู้มีปัญญาแหลมคมดุจลูกศร ซึงหมายถึง ผู้มีปัญญาเฉลียวฉลาด ส่วนชื่อเล่นเดิมบิดาชาวออสเตรียตั้งให้ว่า "แบร้นด์ (Brand)" แต่เนื่องจากอ่านออกเสียงค่อนข้างยาก ครอบครัวของเขาจึงเรียกเพี้ยนมาเป็น แบรี่ หมายถึง "สิ่งที่อยู่บนท้องฟ้า" ณเดชเคยให้สัมภาษณ์กับนิตยสารฉบับหนึ่งว่าเขามีฉายาที่เรียกกันในกลุ่มเพื่อนฝูงคนสนิทว่า "ม้าศึก" เพราะชอบปาร์ตี้ทุกคืนมาตั้งแต่เด็ก และเคยมีรูปร่างปราดเปรียวมาก่อน ณเดชน์เติบโตและอาศัยอยู่ที่ขอนแก่นกับสุดารัตน์ คูกิมิยะ มารดาบุญธรรมชาวอีสานเชื้อสายจีนผู้มีศักดิ์เป็นป้าซึ่งประกอบธุรกิจส่วนตัว และบิดาบุญธรรมชาวญี่ปุ่นคือโยชิโอ คูกิมิยะ เป็นวิศวกรไฟฟ้าซึ่งทำงานในกรุงเทพมหานคร ส่วนบิดาบังเกิดเกล้าเป็นชาวออสเตรีย และมารดาบังเกิดเกล้าเป็นน้องสาวแท้ๆของสุดารัตน์ ณเดชน์เดิมชื่อชลทิศ ยอดประทุม โดยหลังจากโยชิโอและสุดารัตน์รับอุปการะเป็นบุตรบุญธรรม จึงเปลี่ยนชื่อเป็นณเดชน์ คูกิมิยะ ถึงแม้โยชิโอจะเป็นพ่อบุญธรรมของณเดชน์ แต่ณเดชน์ก็ไม่สามารถพูดภาษาญี่ปุ่นได้ เพราะโยชิโอไม่เคยสอน เมื่อสนทนากันโยชิโอจะใช้เพียงภาษาไทยกับภาษาอังกฤษเท่านั้น ณเดชน์เล่าถึงความสัมพันธ์ในครอบครัวว่า โยชิโอและสุดารัตน์มีวิธีการเลี้ยงดูที่ต่างกัน โดยณเดชน์สนิทสนมคุ้นเคยกับสุดารัตน์มากที่สุด เพราะอยู่ใกล้ชิด โทรศัพท์ถึงกันทุกวัน ดูแลถามไถ่เรื่องอาหารสุขภาพ ไม่ดุ ส่วนโยชิโอจะคุยกันแบบผู้ชาย เรื่องวางแผนในอนาคต อาชีพการงาน และเรื่องผู้หญิง ทั้งนี้พ่อและแม่ที่แท้จริงของณเดชน์ได้แยกทางกันไปตั้งแต่ณเดชน์ยังเด็ก ๆ และณเดชน์ก็ไม่เคยเจอหน้าพ่อแท้ ๆ เลย ซึ่งณเดชน์สงสัยที่มาที่ไปของตัวเองมาตั้งแต่เรียนอยู่ชั้นอนุบาล จนกระทั่ง ม.2 หลังโดนเพื่อนทักว่าทำไมหน้าตาไม่คล้ายคนญี่ปุ่น ณเดชน์จึงมาถามความจริงกับสุดารัตน์จนทราบเรื่อง โดยณเดชน์ก็ยอมรับเรื่องดังกล่าว อีกทั้งยังรักและเคารพพ่อและแม่บุญธรรมเหมือนเดิม พร้อมกับยกย่องพ่อบุญธรรมชาวญี่ปุ่นเป็นฮีโร่ในดวงใจ และถือว่าตัวเองก็เป็นลูกครึ่งญี่ปุ่น ณเดชน์เป็นคนชื่นชอบการฟังเพลงและเล่นดนตรีหลายอย่าง เช่นกีตาร์, อูกูเลเล โดยเมื่อว่างจากการถ่ายทำละครเขามักจะนำอูกูเลเล คอร์เน็ต และแบนโจมาเล่นบ่อย ๆ สมัยที่เรียนมัธยมต้นณเดชน์กับเพื่อนได้รวมกลุ่มก่อตั้งวงดนตรีเพื่อแสดงในกิจกรรมต่าง ๆ ของโรงเรียนชื่อวง ดีเอกซ์ โดยณเดชน์เป็นมือเบส และได้ร้องนำบ้างในบางโอกาส รวมถึงชื่นชอบการถ่ายภาพเป็นงานอดิเรก ส่วนกีฬาที่นิยมเล่นในเวลาว่างคือ ฟุตบอล, ว่ายน้ำ, กอล์ฟ ซึ่งโยชิโอสอนให้เมื่ออายุ 10 ปี และเทควันโด ซึ่งเคยเข้าแข่งขันได้รับรางวัลรองชนะเลิศในรุ่นเยาวชนชายอายุไม่เกิน 6-8 ปี จากรายการชิงแชมป์ประเทศไทยเมื่อปี พ.ศ. 2545 นอกจากนี้ณเดชน์ยังมีความศรัทธาและเข้าร่วมกิจกรรมทางพระพุทธศาสนาอย่างสม่ำเสมอ ได้ผ่านการบวชเณรมาเมื่อวัยเยาว์ จนกระทั่งได้รับรางวัลบุคคลผู้มีคุณธรรมส่งเสริมพระพุทธศาสนาเนื่องในวันวิสาขบูชา ประจำปี พ.ศ. 2554 โดยสภาศิลปินส่งเสริมพระพุทธศาสนาแห่งประเทศไทยการศึกษา การศึกษา. ณเดชน์เรียนอนุบาลที่โรงเรียนอนุบาลพิมานเด็ก ต่อชั้นประถมศึกษาที่โรงเรียนมหาไถ่ศึกษาชาย (ปัจจุบันเป็น โรงเรียนมหาไถ่ศึกษาภาคตะวันออกเฉียงเหนือ) กระทั่ง ป.5 ได้ย้ายไปเรียนที่โรงเรียนขอนแก่นวิเทศศึกษา จังหวัดขอนแก่น จนจบชั้นมัธยมศึกษาตอนปลาย (สายวิทย์-คณิต) ระหว่างศึกษาได้เป็นตัวแทนโรงเรียนไปแข่งขันหลายครั้ง เมื่อเข้าสู่ระดับอุดมศึกษาณเดชน์ได้คัดเลือกเข้าศึกษาต่อใน คณะศิลปศาสตร์ มหาวิทยาลัยมหิดล หลักสูตรนานาชาติ แต่ไม่ผ่านการสอบคัดเลือก ณเดชน์จึงได้เลือกเรียนสาขาใหม่ในมหาวิทยาลัยเอกชน เนื่องจากขณะที่ศึกษาอยู่ชั้น ม.4 เขาเริ่มสนใจการผลิตภาพยนตร์สั้น รายการโทรทัศน์ จึงต้องการเรียนรู้การทำงานเบื้องหลัง เช่น กำกับการแสดง และสนใจศึกษาทฤษฎีต่าง ๆ เกี่ยวกับการถ่ายภาพนิ่ง ไม่ชอบการเรียนคณิตศาสตร์ ประกอบกับระยะนั้นเขาได้เข้าสู่วงการบันเทิงเป็นที่เรียบร้อย และเริ่มแสดงละครโทรทัศน์ จึงเลือกเข้าศึกษาต่อในสาขาวิชาการภาพยนตร์และวีดิทัศน์ วิทยาลัยนิเทศศาสตร์ มหาวิทยาลัยรังสิต จวบจนณเดชน์สำเร็จการศึกษาและได้รับเกียรตินิยมอันดับ 2 ด้วยเกรดเฉลี่ย 3.31 ปัจจุบัน ณเดชน์กำลังศึกษาต่อในระดับปริญญาโท วิทยาลัยนิเทศศาสตร์ มหาวิทยาลัยรังสิต ดังเดิม ครั้งเมื่อณเดชน์กำลังศึกษาอยู่ชั้นมัธยม เขาเคยเป็นอาสาสมัครเข้าร่วมโครงการค่ายอาสาชนบทที่โรงเรียนแห่งหนึ่งอยู่ในเขตนอกอำเภอเมือง เป็นโรงเรียนประถมศึกษาขนาดเล็ก ณเดชน์ได้ดูแลและช่วยสอนภาษาอังกฤษให้กับเด็กนักเรียนในที่แห่งนั้นการทำงาน การทำงาน. เมื่อปี พ.ศ. 2551 ผู้จัดการนักแสดง ศุภชัย ศรีวิจิตร ได้ไปเยี่ยมบ้านของศุกลวัฒน์ คณารศ ที่ขอนแก่น และพบณเดชน์ซึ่งอาศัยในหมู่บ้านเดียวกันโดยบังเอิญ จึงร้องขอต่อสุดารัตน์ให้บุตรบุญธรรมเข้าร่วมเป็นนักแสดงในสังกัด ซึ่งเธอก็ยินยอม โดยก่อนหน้านั้นศุภชัยได้รู้จักกับอาจารย์ของณเดชน์ รวมทั้งเคยเห็นภาพณเดชน์จากอินเทอร์เน็ตมาบ้าง แต่เนื่องจากณเดชน์ยังมีอายุเพียง 15 ปี และกำลังศึกษาอยู่ชั้น ม.4 เขาใช้เวลาฝึกบุคลิกภาพ ความสามารถต่าง ๆ ด้านการแสดงเป็นเวลา 2 ปีก่อนที่จะพาเข้าสู่การทำงานในวงการ ปัจจุบันณเดชน์มีบ้านของตัวเองที่กรุงเทพมหานครแล้ว ในตอนแรกณเดชน์ไม่มีความคิดอยากที่จะเป็นนักแสดง เพราะเคยมีอคติส่วนตัวต่อวงการบันเทิงไม่ชอบดูละคร แต่ด้วยคำแนะนำของสุดารัตน์และศุภชัยจึงได้เข้าสู่วงการดารานักแสดงทางฝั่งฮอลลีวูดที่ณเดชน์ชื่นชอบคือ นิโคล คิดแมน เพราะดวงตามีเสน่ห์ มองแล้วรู้สึกประทับใจ โดยให้เหตุผลว่า "ผมชอบเวลามองผู้หญิง แล้วเขามองกลับมาครับ เพราะตาของผู้หญิงแต่ละคนสามารถบอกได้ว่า เขาคิดยังไงกับเรา" แม้ณเดชน์มีภาพลักษณ์ที่สนุกสนานร่าเริง แต่เคยให้สัมภาษณ์ว่า บางครั้งมีโลกส่วนตัวสูง ไม่ต้องการพบปะผู้คน และต้องปรับตัวในการเข้าสู่วงการบันเทิงมาก เมื่อเข้าเป็นนักแสดงในสังกัดของศุภชัยแล้ว ณเดชน์จึงเริ่มงานด้านการเดินแบบเป็นครั้งแรกในงานการกุศลของศูนย์ศิลปาชีพบางไทร ที่ศูนย์การค้าเซ็นทรัลเวิลด์ แล้วจากนั้นก็เริ่มมีผลงานถ่ายแบบให้กับนิตยสารหลายฉบับ โดยครั้งแรกที่ทำงานในวงการบันเทิง ณเดชน์ อายุ 17 ปี กำลังเรียนอยู่ชั้น ม.5 และผลงานแรกที่ปรากฏแพร่ภาพทางวิทยุโทรทัศน์คือภาพยนตร์โฆษณาหมากฝรั่ง ไทรเด้นท์ รีแคลเดนท์ คู่กับพัชราภา ไชยเชื้อ ต่อมาในปี พ.ศ. 2552 ทางสถานีวิทยุโทรทัศน์ไทยทีวีสีช่อง 3 ได้มีการคัดเลือกนักแสดงขึ้น โดยณเดชน์ผ่านการคัดเลือกให้เข้ามาเป็นนักแสดงหน้าใหม่ เพื่อร่วมแสดงในละครโทรทัศน์ เงารักลวงใจ เป็นเรื่องแรก และในปี พ.ศ. 2553 ได้มีผลงานละครที่สร้างชื่อเสียงคือ ดวงใจอัคนี เกมร้ายเกมรัก ความนิยมจากการแสดงละครโทรทัศน์ส่งผลให้ณเดชน์ได้รับเลือกให้เป็นผู้นำเสนอในภาพยนตร์โฆษณาสินค้าเพิ่มขึ้นมากมายจากผลงานโฆษณาที่มีอยู่เป็นจำนวนมากของณเดชน์ทำให้โพสต์ทูเดย์กล่าวว่า ณเดชน์คือ "แชมป์พรีเซ็นเตอร์" รวมทั้งชื่อเสียงในการแสดงละครคู่กับอุรัสยา เสปอร์บันด์ ทำให้ได้นำเสนอภาพยนตร์โฆษณาต่าง ๆ ร่วมกันหลายเรื่องจนกระทั่งถูกเรียกให้เป็นพระนางคู่ขวัญกันทั้งสองคนถูกนำชื่อไปเป็นเรื่องราวสมมุติในงานจิตรกรรมฝาผนังร่วมสมัย เช่น ภาพพิธีมงคลสมรสที่วัดลำปางกลางตะวันออก แสดงวิถีการดำเนินชีวิตและขนบธรรมเนียมประเพณีวัฒนธรรมของคนภาคเหนือ เพื่อบันทึกไว้ให้คนรุ่นหลังศึกษาสืบทอดกันต่อไป ตามมาด้วยผลงานด้านอื่น ๆ เช่นการพากย์การ์ตูนแอนิเมชันสำหรับเยาวชน เรื่อง "ซุปเปอร์ฮีโร่ หล่อช่วยได้" หนึ่งในตัวละครหลักร่วมกับปกรณ์ ฉัตรบริรักษ์ และปริญ สุภารัตน์ ออกอากาศทางช่อง 3 ไปจนถึงการร่วมกิจกรรมทางสังคม ต่าง ๆ เช่น คณะทูตของโครงการรณรงค์ต้านภัยมะเร็งเต้านม 2554 โดยสภากาชาดไทย เป็นพรีเซ็นเตอร์รณรงค์เชิญชวนชายไทยคัดเลือกทหารของกองทัพบกไทย ประจำปี 2555 ในปี 2554-2555 ได้รับเลือกจากมหาวิทยาลัยมหาจุฬาลงกรณราชวิทยาลัย ให้เป็นทูตพระพุทธศาสนาวิสาขบูชานานาชาติ เป็นต้น ซึ่งผลงานหลากหลายประเภทที่ทำไว้ในข้างต้นเป็นจำนวนมาก บางกอกโพสต์จึงเรียกณเดชน์ว่า "Mr Everywhere"" โดยเหตุผลของหนังสือพิมพ์มาจากการที่ประชาชนสามารถพบเห็นณเดชน์ตามแหล่งสื่อต่าง ๆ ต่อเนื่องอยู่ตลอดเวลา ในวันที่ 12 มกราคม พ.ศ. 2555 รายการ เช้าดูวู้ดดี้ ออกอากาศทางช่องโมเดิร์นไนน์ทีวี ได้มีการถ่ายทอดเทปบันทึกเรื่องราวของ "ด.ญ.พรสุภาดา คำกำพุทธ" มีชื่อเล่นว่า "มอมแมม" ที่อาการดีขึ้นอย่างเป็นปรากฏการณ์คล้ายปาฏิหาริย์ หลังจากการดูละคร "เกมร้ายเกมรัก" และชื่นชอบตัวละคร "สายชล" พระเอกของเรื่องรับบทโดยณเดชน์ เป็นแรงบันดาลใจทำให้ปฏิกิริยาของร่างกายฟื้นจากอาการป่วยเร็วขึ้นทุกครั้งก่อนได้รับการผ่าตัด โดยบิดาของเด็กหญิงได้กล่าวไว้ในข้างต้นว่า "ในช่วงเขาป่วยเข้าโรงพยาบาลก็จะไปดูแลเขาตลอด ต้องเขาผ่าตัดมา 3 ครั้งแล้ว ครั้งล่าสุดเมื่อวันที่ 14 ธันวาคม ก็ยังไม่ได้ออกจากโรงพยาบาลเลย เกือบเสียชีวิตไปแล้วเมื่อ 2 ครั้งที่แล้ว แต่ก็รอดมาได้อย่างหวุดหวิด ซึ่งทุกครั้งก็จะทำใจไว้ล่วงหน้า เพราะอาการหนักมากจริง ๆ แต่ครั้งล่าสุดเขาบอกว่าเขาจะกลับมาหาสายชล ซึ่งผมก็ไม่รู้ว่าคืออะไร มารู้ที่หลังว่าเขาชอบดูละครเรื่องนี้ และชอบสายชลมาก คุณหมอบอกว่าอาการดีขึ้นทุกครั้งที่ได้ดูสายชล" โดยมอมแมมเป็นเด็กหญิงวัย 7 ขวบ มีหัวใจเพียงแค่ 2 ห้อง และไม่มีเส้นเลือดไปเลี้ยงที่ปอด ทำให้ไม่สามารถใช้ชีวิตเหมือนเด็กทั่วไปได้ หลังจากที่ณเดชน์ทราบเรื่องจึงเดินทางมาพบเด็กหญิงเพื่อมาให้กำลังใจพบปะพูดคุยกับเด็กหญิง เมื่อวันที่ 19 มกราคม ที่สถาบันสุขภาพเด็กแห่งชาติมหาราชินี บริจาคเงินให้ครอบครัวของเด็กหญิงอีก 50,000 บาท วันรุ่งขึ้นเป็นพาดหัวข่าวใหญ่บนหน้าหนึ่งหนังสือพิมพ์หลายฉบับ และรายการโทรทัศน์บันเทิงหลายช่อง จนวันที่ 22 มกราคม เด็กหญิงสามารถออกจากโรงพยาบาล และกลับมาเรียนหนังสือได้อย่างเป็นปกติ การจัดงานวันแม่แห่งชาติปี 2555 ณเดชน์ได้รางวัลลูกที่มีความกตัญญูกตเวทีอย่างสูงต่อแม่ จากสภาสังคมสงเคราะห์แห่งประเทศไทย ในพระบรมราชูปถัมภ์ มีจำนวนทั้งสิ้น 85 คน จาก 330 คน ซึ่งณเดชน์เป็นหนึ่งในผู้ที่ได้รับรางวัลประเภท นักร้อง นักแสดง ศิลปิน โดยเข้ารับพระราชทานโล่ประกาศเกียรติคุณจากพระเจ้าวรวงศ์เธอ พระองค์เจ้าโสมสวลี พระวรราชาทินัดดามาตุ ณ อาคารใหม่ สวนอัมพร ในปีต่อมาณเดชน์ได้รับประกาศเกียรติคุณเป็นทูตพระพุทธศาสนาวิสาขบูชานานาชาติ ประจำปี พ.ศ. 2556 และการประกาศเกียรติคุณรางวัลครอบครัวชาวพุทธมามกะ จากทางมหาวิทยาลัยมหาจุฬาลงกรณราชวิทยาลัย เมื่อปี พ.ศ. 2555 มีผลงานแสดงละครเรื่อง ธรณีนี่นี้ใครครอง แสดงร่วมกับอุรัสยา เสปอร์บันด์อีกหน การแสดงของทั้งคู่กรุงเทพธุรกิจวิจารณ์ไว้ว่า "ได้เผยถึงความหลากอารมณ์ มีหลายอย่างปน ๆ กันอยู่ แล้วปล่อยออกมาแบบกระตุ้นการรับรู้" (sensory stimulus) นอกจากงานแสดงแล้ว ณเดชน์ยังได้มีส่วนร่วมกำกับภาพยนตร์กับเทศกาลภาพยนตร์นานาชาติเชียงคาน ซึ่งเป็นผลงานการกำกับครั้งแรก และปีเดียวกันนี้ ณเดชน์ได้มีงานแสดงภาพยนตร์เรื่องแรกในชีวิตคือ คู่กรรม รับบทเป็น "โกโบริ" ทหารญี่ปุ่น ซึ่งณเดชน์ได้เข้าเรียนภาษาญี่ปุ่นเพิ่มเพื่อการพูดภาษาไทยให้มีสำเนียงเหมือนชาวญี่ปุ่น แสดงร่วมกับอรเณศ ดีคาบาเลส นักแสดงหน้าใหม่ กำกับภาพยนตร์โดยกิตติกร เลียวศิริกุล ผลิตโดยเอ็ม เทอร์ตี้ไนน์ ทางด้านผู้จัดจันทิมา เลียวศิริกุล กล่าวถึงเหตุผลที่เลือกณเดชน์ เพราะโกโบริคือณเดชน์ ว่า "เพราะเขาคือคนที่เหมาะที่สุด เราไม่เคยคิดจะเปลี่ยนตัวเป็นคนอื่น และถ้าไม่ได้น้องเขาจริง ๆ โปรเจกต์นี้ก็คงต้องเก็บไว้ก่อน" ในปี พ.ศ. 2556 ณเดชน์ได้รับเลือกให้แสดงในละครเรื่อง "รอยฝันตะวันเดือด" จากละครซีรีส์ชุด "The Rising Sun" รับบทเป็นคนญี่ปุ่นชื่อ ริว แสดงคู่กับคู่ขวัญ อุรัสยา เสปอร์บันด์ ออกอากาศในปี พ.ศ. 2557 และยังได้ร้องเพลงประกอบละครร่วมกัน ในเพลง แล้วเราจะได้รักกันไหม ซึ่งเพลง "แล้วเราจะได้รักกันไหม" ติดอันดับสูงสุดที่ 1 จากการจัดอันดับของซี้ดเอฟเอ็มในชาร์ตของ ซี้ดเอฟเอ็ม ชาร์ตท็อป 20 ประจำวันที่ 14 กันยายน พ.ศ. 2557 โดยวัดจากการออกอากาศของคลื่นวิทยุในกรุงเทพมหานครและปริมณฑล จำนาน 40 สถานี และละครซีรีส์ชุดนี้ยังได้รับรางวัล สเปเชียล อวอร์ด จากองค์การส่งเสริมการท่องเที่ยวแห่งประเทศญี่ปุ่น เนื่องจากซีรีส์ชุดนี้ได้ถ่ายทอดสถานที่ท่องเที่ยวและวัฒนธรรมญี่ปุ่นอย่างสวยงาม จนมีนักท่องเที่ยวเดินทางตามรอยสถานที่ต่าง ๆ ในปี [[พ.ศ. 2557]] แสดงละครเรื่อง "[[ลมซ่อนรัก]]" การพบกันครั้งแรกกับ [[ณฐพร เตมีรักษ์ |แต้ว ณฐพร เตมีรักษ์]] จะออกอากาศในปี [[พ.ศ. 2558]] และยังได้รับบทฝาแฝดเรื่องแรกของณเดชน์อีกด้วย และปลายปีเดียวกันณเดชน์มีละครเรื่อง "[[ตามรักคืนใจ]]" แสดงนำร่วมกับ [[นิษฐา จิรยั่งยืน|มิว นิษฐา จิรยั่งยืน]]ผลงานรางวัลภาพลักษณ์ผลการสำรวจความนิยมฉายาในวงการบันเทิงภาพลักษณ์. ฉายาในวงการบันเทิง. - [[พ.ศ. 2553]] - ได้รับฉายา "ซุป\'ตาร์พันธุ์ข้าวเหนียว" จากสมาคมนักข่าวบันเทิงประจำปี 2553 - [[พ.ศ. 2554]] - ได้รับฉายา "แบรี่ ฮอตเว่อร์" จากนิตยสาร สตาร์นิวส์ - ได้รับฉายา "เทพบุตรลูกทุ่ง" จากนิตยสาร สาระแนดารา - ได้รับฉายา "วีรบุรุษไหปลาแดก" จากนิตยสาร ทีวีอินไซต์ผู้ทรงอิทธิพลในวงการบันเทิงผู้ทรงอิทธิพลในวงการบันเทิง. - [[พ.ศ. 2554]] - อันดับ 7 จาก 12 ผู้ทรงอิทธิพลที่สุดแห่งวงการบันเทิงปี 2011 โดยนิตยสาร Request ฉบับเดือนมกราคม 2555 - [[พ.ศ. 2555]] - ได้รับการยกย่องให้เป็น 1 ใน 6 "ดาราแห่ง[[ศตวรรษ]]" และ 1 ใน 12 "คนบันเทิงรวย ผู้ทรงอิทธิพลครึ่งปีแรก" โดยนิตยสาร Request - ได้รับการยกย่องให้เป็น ดาราทรงอิทธิพลในวงการบันเทิง โดยหนังสือพิมพ์ ดาราเดลี่ - อันดับ 1 จาก 12 พระเอกทรงอิทธิพล รายได้ต่อวันสูงที่สุดแห่งปี โดยนิตยสาร Request - ได้รับการยกย่องให้เป็น 1 ใน 5 ผู้ทรงอิทธิพลของวงการบันเทิงแห่งปี 2555 โดยรายการ [[ไนน์เอ็นเตอร์เทน]] ทาง[[สถานีโทรทัศน์โมเดิร์นไนน์]] - ได้รับการยกย่องให้เป็น บุคคลบันเทิงแห่งปี 2555 โดยสถานีโทรทัศน์[[วอยซ์ทีวี]] - 1 ใน 10 ผู้ทรงอิทธิพลของวงการบันเทิงปี 2555 สาขา ซูเปอร์สตาร์พระเอกผู้ทรงอิทธิพล โดยรายการ บันเทิงทูเดย์ ทางมีเดียแชนแนลสื่อบันเทิงต่าง ๆสื่อบันเทิงต่าง ๆ. - [[พ.ศ. 2553]] - อันดับ 2 "หน้าใหม่เน็ตล่ม" จากละครเรื่อง "[[ดวงใจอัคนี_ (ละครโทรทัศน์)|ดวงใจอัคนี]]" - อันดับ 1 ดาวรุ่งมาแรงแห่งปี จากนิตยสาร Oops! - อันดับ 1 ยังบลัดมาแรงแห่งปี 2010 จากนิตยสาร ทีวีพูล - อันดับ 1 ดาวรุ่งพุ่งแรงปี 53 จากสำนักข่าว ดาราเดลี่ - อันดับ 1 ดาวรุ่งปีเสือ (2553) จากสำนักข่าว INN News - [[พ.ศ. 2554]] - อันดับ 1 คู่ขวัญแห่งปี ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" จาก จีเมมเบอร์ ในเครือบริษัท[[แกรมมี่]] - ดาราคู่ขวัญแห่งปี 2554 ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" จากสถานีโทรทัศน์[[วอยซ์ทีวี]] - ได้รับตำแหน่ง "เจ้าพ่อพรีเซ็นเตอร์" แห่งปี 2554 จากสำนักข่าว [[เอเอสทีวีผู้จัดการ]] - อันดับ 1 จาก 5 พระเอกสุดแซ่บ สยบโลกออนไลน์ ใน Big Fan Can Vote Season 1 จากการโหวตโดย มายทรีสเปซ ทาง[[ไทยทีวีสีช่อง 3]] - อันดับ 1 จาก 5 คู่ขวัญวันเดอร์แลนด์ ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" ใน Big Fan Can Vote Season 1 จากการโหวตโดย มายทรีสเปซ ทาง[[ไทยทีวีสีช่อง 3]] - อันดับ 4 ดาราชายยอดนิยมแห่งปี 2554 โดย Vote Everything - อันดับ 1 จาก 5 พระเอกฮอตที่คนชอบมากสุด ปี 2010 Hot Shot ช่วง Star Post ทาง[[ไทยทีวีสีช่อง 3]] - ได้รับการยกย่องให้เป็นผู้ชายประจำปี 2011 ขึ้นปกนิตยสาร "ดิฉัน" ฉบับที่ 823 มิถุนายน 2554 - อันดับ 1 จาก 10 ว่าที่ซุปตาร์มายาอนาคตไกล จากนิตยสาร สตาร์นิวส์ ฉบับที่ 378 ประจำวันที่ 25-31 กรกฎาคม พ.ศ. 2554 - อันดับ 1 จาก 10 อันดับแรกดาราที่รายได้รวมจากงานโฆษณามากที่สุดประจำปี 2554 โดยนิตยสาร รีเควส ฉบับเดือนกันยายน 2554 - อันดับ 2 หนุ่มน่ากอดแห่งปี 2011 จากนิตยสาร [[สุดสัปดาห์ (นิตยสาร)|สุดสัปดาห์]] - อันดับ 1 พระเอกสุดฮอต จากตัวละคร "สายชล" จากสำนักข่าว [[สยามดารา]] - ได้รับการยกย่องให้เป็น ดารา-บันเทิง ที่สุดแห่งปี 2554 ฐานะ "บุคคลแห่งปี" จากรายการ [[ไนน์เอ็นเตอร์เทน]] ทาง[[สถานีโทรทัศน์โมเดิร์นไนน์]]- [[พ.ศ. 2555]] - อันดับ 3 ดิ เออร์เบิน แอพพาเรล คูลสตาร์ อวอร์ด 2011 - ได้รับการยกย่องให้เป็น "เจ้าพ่อพรีเซ็นเตอร์" แห่งปี 2554 โดยหนังสือพิมพ์ ดาราเดลี่ - 1 ใน 14 พระนางฮอตฮิตน่าจับตาปี 2555 จาก true - อันดับ 1 จาก 5 อันดับ คู่พระนางเคมีลงตัวที่สุด ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" จากรายการ บันเทิง ๕ หน้า ๑ ทาง[[ททบ.5]] - อันดับ 1 จาก 5 ผลโหวตสุดยอดนิตยสาร ประจำปี 55 สาขาฉบับที่น่าประทับใจที่สุด (VOLUME vol. 7 no. 168 พฤษภาคม 2012 คะแนน 20.77% ปก ณเดชน์ คูกิมิยะ, ญาญ่า-อุรัสยา เสปอร์บันด์ ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" โดย Magazinedee Award - อันดับ 1 จาก 10 หนุ่มน่ากอดแห่งปี 2012 จากนิตยสาร [[สุดสัปดาห์ (นิตยสาร)|สุดสัปดาห์]] - อันดับ 1 พระเอกแห่งปี 2554 (ร้อยละ 51.21) จากละครเรื่อง "[[เกมร้ายเกมรัก]]" จากรายการ [[เรื่องเล่าเช้านี้]] ทาง[[ไทยทีวีสีช่อง 3]] - อันดับ 1 เพลง "[[ให้รักมันโตในใจ]]" เพลงไทยยอดนิยมจากการจัดอันดับของบริษัท ดิจิตอล แอสโซซิเอทส์ จำกัด ในสัปดาห์ที่ 34,35 ของปี พ.ศ. 2555 ในวันที่ 19 สิงหาคม - 1 กันยายน โดยวัดจากการออกอากาศของคลื่นวิทยุในกรุงเทพมหานครและปริมณฑลจำนาน 40 สถานี - อันดับ 1 Best of 2011 Focus 2012 พระเอกโดนใจแห่งปี จากสำนักข่าว [[สยามดารา]] - ได้รับการยกย่องให้เป็น Brand Essence "ณเดชน์เอสเซนส์ ดีเอ็นเอ เมกะซุป\'ตาร์" จากนิตยสาร Positioning - Big Fan Can Vote Season 2 จากการโหวตทาง มายทรีสเปซ โดย [[ไทยทีวีสีช่อง 3]] - อันดับ จาก 5 อันดับ พระเอกชาวขอนแก่น โดยรายการ เกาซุปตาร์ ทางช่องดาราเดลี่ทีวี หนังสือพิมพ์ดาราเดลี่ - อันดับ 1 พระเอกสุดฮอต นางเอกสุดฮิต จากสำนักข่าว นิวส์พลัส - อันดับ 1 พระเอกสุดฮอตแห่งปี 55 จากละครเรื่อง "[[ธรณีนี่นี้ใครครอง]]" โดยหนังสือพิมพ์ [[สยามดารา]] - อันดับ 1 ดาราที่สาว ๆ อยากไปเคาท์ดาวน์ด้วยมากที่สุด สำรวจโดยรายการ The poll ทาง[[ไทยทีวีสีช่อง 3]] - อันดับ 1 Most of Charming Celebrity 2012 จาก Most of the most 2012 โดยนิตยสาร เปรียว- [[พ.ศ. 2556]] - 1 ใน 10 "คู่จิ้นรักจริงทะลุจอ" ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" โดยไทยรัฐออนไลน์ - อันดับ 1 จาก 10 คู่จิ้นดาราแห่งปี 2013 ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" โดยรายการดาราวาไรตี้โพล - อันดับ 3 ดาราที่หนุ่มชอบของขวัญที่มามอบให้ในวันคริสต์มาส โดยรายการ The Poll - อันดับ 1 ดาราที่สาวๆ ชอบของขวัญที่มามอบให้ในวันคริสต์มาส โดยรายการ The Poll - อันดับ 1 ดาราที่อยากให้เป็นแซนต้ามามอบของขวัญให้ในวันคริสต์มาส (ออนไลน์) โดยรายการ The Poll - 1 ใน 10 หนุ่มน่ากอดแห่งปี 2013 จากนิตยสาร [[สุดสัปดาห์ (นิตยสาร)|สุดสัปดาห์]] - อันดับ 4 จาก 5 หน้าหนาวนี้คุณอยากกอดซุปตาร์คนไหนมากที่สุด โดย Star news - อันดับ 1 จาก 5 ดาราที่คนไทยอยากไปลอยกระทงด้วยมากที่สุด ปี 2013 จากรายการ ซิสเตอร์เดย์ ทาง[[ททบ.5]]- [[พ.ศ. 2557]] - อันดับ 1 จาก 10 นักแสดงนำชาย ปี 2556 จาก ละคร ไทยทีวีสี ช่อง 3 (Ch3\'s Drama) - อันดับ 1 จาก 5 ดาราชายที่สาวๆอยากไปลอยกระทงด้วยมากที่สุด โดยรายการดาราวาไรตี้โพล - อันดับ 1 ซุปตาร์สุดแซ่บที่อยากเห็นขึ้นปก Oops! ) ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" โดยรายการดาวกระจาย - อันดับ 1 จาก 10 คู่จิ้นแห่งปี 2014 ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" โดยรายการซุปตาร์หน้า 1 - อันดับ 1 จาก 5 คู่จิ้นฟินจิกหมอน ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" โดยรายการดาราวาไรตี้โพล - อันดับ 5 จาก 5 ซุปตาร์ฉาวฆ่าไม่ตาย โดยรายการดาราวาไรตี้โพล - อันดับ 4 จาก 5 ดาราใจบุญ ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" โดยรายการดาราวาไรตี้โพล- [[พ.ศ. 2558]] - อันดับ 4 จาก 5 นักแสดงนำชาย ปี 2557 จาก ละคร ไทยทีวีสี ช่อง 3 (Ch3\'s Drama) - อันดับ 1 จาก 5 ซุปตาร์ฮอตแห่งปี 2557 โดยรายการดาราวาไรตี้โพล - อันดับ 5 จาก 5 ดาราชายแต่งหญิงสวยแจ่ม โดยรายการดาราวาไรตี้โพล - อันดับ 1 ซุปตาร์ที่อยากให้มาติดหัวใจ ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" โดยรายการดาวกระจาย - อันดับ 1 จาก 5 ดาราแฟนคลับรักเหนียวแน่น ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" โดยรายการดาราวาไรตี้โพล - อันดับ 1 จาก 5 ซุปตาร์โนโซเชียล โดยรายการดาราวาไรตี้โพล - อันดับ 5 จาก 5 คู่รักซุปตาร์หวานฉ่ำ ร่วมกับ "[[อุรัสยา เสปอร์บันด์]]" โดยรายการดาราวาไรตี้โพล - อันดับ 1 จาก 5 พระเอกลูกครึ่งสุดฮอต โดยรายการดาราวาไรตี้โพล - อันดับ 4 จาก 5 ซุปตาร์ค่าตัวแพง โดยรายการดาราวาไรตี้โพล - อันดับ 2 จาก 10 ดาราชายที่คนอยากเห็นโชว์หุ่นซัมเมอร์นี้ โดยไทยรัฐออนไลน์ - อันดับ 2 จาก 5 ดาราหนุ่มน่าปะแป้ง โดยรายการดาราวาไรตี้โพล - อันดับ 1 จาก 5 ดาราบทฝาแฝดอินสุดๆ โดยรายการดาราวาไรตี้โพล - อันดับ 1 จาก 5 ดราม่าดาราดัง โดยรายการดาราวาไรตี้โพล - อันดับ 1 จาก 5 พระ-นางเสียงเป๊ะ โดยรายการดาราวาไรตี้โพล - อันดับ 2 จาก 5 คู่จิ้นหวานฉ่ำนอกจอ โดยรายการดาราวาไรตี้โพล - อันดับ 1 จาก 3 ดาราช่อง 3 คนไหนที่เป็นที่หนึ่งในใจคุณตลอดมา (ออนไลน์) โดยรายการ The Poll - อันดับ 1 จาก 5 คู่หวานปากแข็ง โดยรายการดาราวาไรตี้โพล - อันดับ 1 จาก 3 ดาราหรือพิธีกรช่อง 3 คนไหนที่คุณอยากไปเวียนเทียนเป็นพิเศษ (ออฟไลน์) โดยรายการ The Poll- 1 ใน 10 ซุปตาร์หนุ่มสุดฮอตปี 2558 โดยสำนักข่าว Newsplus'
questions = [
    'ณเดชน์ คูกิมิยะเป็นนักแสดงและนายแบบลูกครึ่งไทย-ออสเตรีย มีชื่อจริงว่าอะไร',
    'ณเดชน์ คูกิมิยะ เกิดเมื่อวันที่ 17 ธันวาคม พ.ศ. 2534 เป็นนักแสดงและนายแบบลูกครึ่งไทย-ออสเตรีย มีชื่อจริงเรียกว่าอะไร',
    'ณเดชน์ คูกิมิยะ นักแสดงชาย มีชื่อจริงว่าอะไร',
    'ณเดชน์ คูกิมิยะ นักแสดงและนายแบบลูกครึ่งไทย-ออสเตรีย เกิดเมื่อวันที่เท่าไร',
    'ณเดชน์ คูกิมิยะเป็นนักแสดงและนายแบบลูกครึ่งไทย-ออสเตรีย เกิดวันอะไร',
    'ณเดชน์ คูกิมิยะ หรือ ชลทิศ ยอดประทุม เข้าสู่วงการจากการแนะนำ และชักชวนของใคร'
]

# Predict

In [76]:
eval_examples = [Example(context, question) for question in questions]
eval_features = convert_examples_to_features(
            examples=eval_examples,
            tokenizer=bert_tokenizer)

all_input_ids = torch.tensor([f.input_ids for f in eval_features], dtype=torch.long)
all_input_mask = torch.tensor([f.input_mask for f in eval_features], dtype=torch.long)
all_segment_ids = torch.tensor([f.segment_ids for f in eval_features], dtype=torch.long)
all_example_index = torch.arange(all_input_ids.size(0), dtype=torch.long)
eval_data = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_example_index)
# Run prediction for full data
eval_sampler = SequentialSampler(eval_data)
eval_dataloader = DataLoader(eval_data, sampler=eval_sampler, batch_size=8)

In [77]:
model.eval()
all_results = []
RawResult = collections.namedtuple(
    "RawResult",
    ["unique_id", "start_logits", "end_logits"])
for input_ids, input_mask, segment_ids, example_indices in eval_dataloader:
    input_ids = input_ids.to(device)
    input_mask = input_mask.to(device)
    segment_ids = segment_ids.to(device)
    with torch.no_grad():
        batch_start_logits, batch_end_logits = model(input_ids, segment_ids, input_mask)
    for i, example_index in enumerate(example_indices):
        start_logits = batch_start_logits[i].detach().cpu().tolist()
        end_logits = batch_end_logits[i].detach().cpu().tolist()
        eval_feature = eval_features[example_index.item()]
        unique_id = int(eval_feature.unique_id)
        all_results.append(
            RawResult(
                unique_id=unique_id,
                start_logits=start_logits,
                end_logits=end_logits))

In [79]:
results = predict(eval_examples, eval_features, all_results, 20, 3 ,30)

In [85]:
len(results)

6

In [80]:
results[0]

('ณเดชน์ คูกิมิยะเป็นนักแสดงและนายแบบลูกครึ่งไทย-ออสเตรีย มีชื่อจริงว่าอะไร',
 [OrderedDict([('text', 'ชล ทิศ ยอด ประ ทุม'),
               ('probability', 0.999221974251911)]),
  OrderedDict([('text', 'ชล ทิศ ยอด'),
               ('probability', 0.0002875127470499479)]),
  OrderedDict([('text', 'ชล ทิศ'), ('probability', 0.00022322841529103898)]),
  OrderedDict([('text', 'ช'), ('probability', 9.029754028794435e-05)]),
  OrderedDict([('text', 'ชล ทิศ ยอด ประ ทุ'),
               ('probability', 5.306718674291623e-05)]),
  OrderedDict([('text', 'ม'), ('probability', 3.9176208499504085e-05)]),
  OrderedDict([('text', 'สุดา รัตน์'),
               ('probability', 1.5095250841199344e-05)]),
  OrderedDict([('text', 'ทิศ ยอด ประ ทุม'),
               ('probability', 1.3222454553636894e-05)]),
  OrderedDict([('text', 'ชื่อจริง ว่า ชล ทิศ ยอด ประ ทุม'),
               ('probability', 1.2529661305809148e-05)]),
  OrderedDict([('text', 'ยอด ประ ทุม'),
               ('probability', 1.0766307762

In [81]:
results[1]

('ณเดชน์ คูกิมิยะ เกิดเมื่อวันที่ 17 ธันวาคม พ.ศ. 2534 เป็นนักแสดงและนายแบบลูกครึ่งไทย-ออสเตรีย มีชื่อจริงเรียกว่าอะไร\t',
 [OrderedDict([('text', 'ชล ทิศ ยอด ประ ทุม'),
               ('probability', 0.998368168477886)]),
  OrderedDict([('text', 'ชล ทิศ'), ('probability', 0.0009262076807176354)]),
  OrderedDict([('text', 'ชล ทิศ ยอด'),
               ('probability', 0.00022658338502827154)]),
  OrderedDict([('text', 'ชล ทิศ ยอด ประ ทุม เกิด วัน อังคาร'),
               ('probability', 0.00014893980104135847)]),
  OrderedDict([('text', 'ชล'), ('probability', 0.00012291575817414182)]),
  OrderedDict([('text', 'ม'), ('probability', 9.078988060557106e-05)]),
  OrderedDict([('text', 'ช'), ('probability', 4.263770697637343e-05)]),
  OrderedDict([('text', 'ยอด ประ ทุม'),
               ('probability', 1.301265143187407e-05)]),
  OrderedDict([('text', 'ชล ทิศ ยอด ประ ทุ'),
               ('probability', 1.0097297660970601e-05)]),
  OrderedDict([('text', 'ชล ทิศ ยอด ปร'),
               ('prob

In [82]:
results[2]

('ณเดชน์ คูกิมิยะ นักแสดงชาย มีชื่อจริงว่าอะไร',
 [OrderedDict([('text', 'ชล ทิศ ยอด ประ ทุม'),
               ('probability', 0.9989970877434576)]),
  OrderedDict([('text', 'ชล ทิศ ยอด ประ ทุม เกิด วัน อังคาร'),
               ('probability', 0.0003361717534695023)]),
  OrderedDict([('text', 'ม'), ('probability', 0.0001934064298554278)]),
  OrderedDict([('text', 'ประ ทุม'), ('probability', 0.0001001910170432569)]),
  OrderedDict([('text', 'ยอด ประ ทุม'),
               ('probability', 9.27189541915218e-05)]),
  OrderedDict([('text', 'ช'), ('probability', 6.989253635215042e-05)]),
  OrderedDict([('text', 'ชล ทิศ ยอด'),
               ('probability', 6.018808516479943e-05)]),
  OrderedDict([('text', 'ทิศ ยอด ประ ทุม'),
               ('probability', 5.7997589846557184e-05)]),
  OrderedDict([('text', 'ชล ทิศ'), ('probability', 4.058329706795803e-05)]),
  OrderedDict([('text', 'ชล ทิศ ยอด ประ ทุ'),
               ('probability', 2.8989808253883588e-05)]),
  OrderedDict([('text', 'ศ ยอด ปร

In [83]:
results[3]

('ณเดชน์ คูกิมิยะ นักแสดงและนายแบบลูกครึ่งไทย-ออสเตรีย เกิดเมื่อวันที่เท่าไร',
 [OrderedDict([('text', '17'), ('probability', 0.9995713249244411)]),
  OrderedDict([('text', '17 ธันวาคม พ . ศ . 2534'),
               ('probability', 0.00031700824350957684)]),
  OrderedDict([('text', 'เกิด เมื่อ วันที่ 17'),
               ('probability', 4.078065398308326e-05)]),
  OrderedDict([('text', 'อังคาร วันที่ 17'),
               ('probability', 3.358135448504282e-05)]),
  OrderedDict([('text', '17 ธันวาคม'),
               ('probability', 1.3885349579806341e-05)]),
  OrderedDict([('text', 'วันที่ 17'),
               ('probability', 1.0480172252954968e-05)]),
  OrderedDict([('text', 'เมื่อ วันที่ 17'),
               ('probability', 3.412326438178781e-06)]),
  OrderedDict([('text', '2534'), ('probability', 1.992843236735551e-06)]),
  OrderedDict([('text', 'อังคาร วันที่ 17 ธันวาคม พ . ศ . 2534'),
               ('probability', 1.1170299405566972e-06)]),
  OrderedDict([('text', 'วัน อังคาร วันท

In [86]:
results[4]

('ณเดชน์ คูกิมิยะเป็นนักแสดงและนายแบบลูกครึ่งไทย-ออสเตรีย เกิดวันอะไร',
 [OrderedDict([('text', 'อังคาร'), ('probability', 0.9976334253196787)]),
  OrderedDict([('text', 'วัน อังคาร'),
               ('probability', 0.0021318443566365358)]),
  OrderedDict([('text', 'อ'), ('probability', 0.00010809340217791022)]),
  OrderedDict([('text', 'าร'), ('probability', 8.033561251252832e-05)]),
  OrderedDict([('text', 'เกิด วัน อังคาร'),
               ('probability', 3.0204041642140102e-05)]),
  OrderedDict([('text', 'อังคาร วันที่ 17'),
               ('probability', 6.206396349164334e-06)]),
  OrderedDict([('text', 'อังคาร วันที่ 17 ธันวาคม'),
               ('probability', 3.403068059786174e-06)]),
  OrderedDict([('text', 'อังคาร วันที่ 17 ธันวาคม พ . ศ . 2534'),
               ('probability', 2.1304590548554526e-06)]),
  OrderedDict([('text', 'วันวิสาขบูชา'),
               ('probability', 1.022971580050533e-06)]),
  OrderedDict([('text', 'คาร'), ('probability', 7.522617097961928e-07)]),
  

In [87]:
results[5]

('ณเดชน์ คูกิมิยะ หรือ ชลทิศ ยอดประทุม เข้าสู่วงการจากการแนะนำ และชักชวนของใคร',
 [OrderedDict([('text', 'ศุภ ชัย ศรี วิจิตร'),
               ('probability', 0.9968348217092636)]),
  OrderedDict([('text', 'ศุภ ชัย'), ('probability', 0.0023166947489004267)]),
  OrderedDict([('text', 'ผู้จัดการ นักแสดง ศุภ ชัย ศรี วิจิตร'),
               ('probability', 0.00027646602456556373)]),
  OrderedDict([('text', 'ศุภ ชัย ศรี วิจิต'),
               ('probability', 9.774178311766824e-05)]),
  OrderedDict([('text', 'ศรี วิจิตร'),
               ('probability', 9.317431457575332e-05)]),
  OrderedDict([('text', 'ศุภ ชัย ศรี'),
               ('probability', 7.67204715132988e-05)]),
  OrderedDict([('text', 'ภ ชัย ศรี วิจิตร'),
               ('probability', 7.607824987240582e-05)]),
  OrderedDict([('text', 'ชัย ศรี วิจิตร'),
               ('probability', 5.395123601954948e-05)]),
  OrderedDict([('text', 'ศ'), ('probability', 4.9962721436915253e-05)]),
  OrderedDict([('text', 'สุดา รัตน์ คู กิ มิ ยะ